# TFM.

# Agrupar complejos de acuerdo a sus descriptores con K-means

### Secciones

- Algoritmo K-means

In [2]:
import pyodbc

server = 'BIOE-MAJC' 
database = 'bioe' 
username = 'bioe' 
password = 'oe@LoCaL' 
trusted = 'yes;' 

cnxn = pyodbc.connect('Driver={SQL Server};SERVER='+server+';DATABASE='+database+';ENCRYPT=no;'+username+';PWD='+ password)
cursor = cnxn.cursor()

import pandas as pd


#### Algoritmo K-means

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min

%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['figure.figsize'] = (16, 9)
plt.style.use('ggplot')

query = "SELECT * FROM [bioe].[TFM].[tfm_NNaN0_S] ();"

df = pd.read_sql_query(query, cnxn)
print(df.head(5))
#print(df.loc[0,:])
#print(df[["CP_BFKV","CP_BL"]])


      Complex  benchmark  dG_S  CP_BFKV  CP_BL  CP_BT  CP_GKS  CP_HLPL  \
0   1A2K_C:AB          1     1  -0.8654 -6.506  -5.48    -8.0    -28.6   
1    1ACB_E:I          1     1  -2.3756 -6.130  -7.74    -2.4    -34.0   
2   1AHW_AB:C          1     1  -1.0114 -8.650  -4.96   -14.0    -34.2   
3    1AK4_A:D          1     0   1.1344 -2.076  -3.98    -4.0    -12.0   
4  1AKJ_AB:DE          1     0   1.5450 -4.558   2.74   -10.0    -26.8   

   CP_MJPL  CP_MJ3h  ...  AP_PISA  FIREDOCK  FIREDOCK_AB  FIREDOCK_EI  \
0   -140.8    -2.96  ...  -0.2445    -58.02       -71.00       -25.76   
1   -178.0   -11.50  ...  -0.4878    -88.88       -96.94       -62.16   
2   -196.2     2.36  ...  -0.0288    -36.54       -67.22        -3.33   
3   -122.4    -2.90  ...  -0.2250    -42.73       -47.67       -29.09   
4   -191.2     4.64  ...  -0.2936    -45.19       -67.13        -4.81   

     CP_PIE   CP_DDG_U   CP_DDG_W   AP_DDG_U   AP_DDG_W      DDG_V  
0  1.138510  -4.560318  -3.938181  -2.889758 -1

C:\Users\coept\AppData\Local\Temp\ipykernel_13884\550404047.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, cnxn)


In [9]:
df.describe()
print(df.groupby("dG_S").size())


dG_S
0     53
1    125
dtype: int64


In [42]:
#X = np.array(df[["CP_BFKV","CP_BL"]])
X = np.array(df[["CP_BFKV","CP_BL","CP_BT","CP_GKS","CP_HLPL","CP_MJPL","CP_MJ3h","CP_MJ2h","CP_MJ1","CP_MJ2","CP_MSBM","CP_MS","CP_Qa","CP_Qm","CP_Qp" \
  ,"CP_RO","CP_SKOb","CP_SKOa","CP_SJKG","CP_TD","CP_TEl","CP_TEs","CP_TS","CP_VD","CP_SKOIP","AP_DCOMPLEX","AP_dDFIRE","AP_DFIRE2","CP_RMFCEN1" \
  ,"CP_RMFCEN2","CP_RMFCA","CP_TB","CP_TSC","AP_T1","AP_T2","AP_DOPE","AP_DOPE_HR","AP_ACE","INSIDE","HBOND","PI_PI","CAT_PI","ALIPH","ZRANK","ZRANK2" \
  ,"ROT_S","TRANS_S","NIPacking","NSC","FA_ATR","FA_REP","LK_SOLV","FA_PP","CG_VDW","CG_PP","CG_ENV","CG_BETA","HBOND2","AA_PROP","ROSETTADOCK","NHB" \
  ,"ELE","DESOLV","VDW","PYDOCK_TOT","ODA","PROPNSTS","SIPPER","AP_OPUS_PSP","AP_GEOMETRIC","AP_DARS","AP_URS","AP_MPS","AP_W1","CP_D1","AP_calRW" \
  ,"AP_calRWp","AP_PISA","FIREDOCK","FIREDOCK_AB","FIREDOCK_EI","CP_PIE","CP_DDG_U","CP_DDG_W","AP_DDG_U","AP_DDG_W","DDG_V"]])
y = np.array(df["dG_S"])

print(X)
print(y)

X.shape

[[ -0.8654      -6.506       -5.48       ...  -2.889758   -10.427095
  -20.76623326]
 [ -2.3756      -6.13        -7.74       ...  -9.272135   -20.305172
  -24.25842593]
 [ -1.0114      -8.65        -4.96       ... -25.095661   -33.166934
  -23.90383239]
 ...
 [ -0.9606       2.55        -3.26       ...  -7.067192   -18.379305
  -23.96102984]
 [ -3.7898      -2.402       -7.84       ... -10.058169   -13.256339
  -19.02188746]
 [  1.9632      -4.45         1.02       ...  -6.593052   -14.494473
   -8.75725307]]
[1 1 1 0 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1 0 0 1 0 1 1 1 1 0 1 1 0 1 0 1 1 1 0
 1 1 0 1 1 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 0 1 1 0 1 0 1 1 0
 1 1 1 0 0 1 1 1 1 1 1 1 0 1 0 0 1 0 1 0 1 1 1 0 0 0 1 1 0 1 1 0 1 1 1 0 1
 1 0 1 0 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 0 0 1 1 1 0 1 1 1 0 1 1 1 0 0 1 1
 1 1 1 0 0 1 0 1 0 0 1 1 0 0 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 0]


(178, 87)

In [39]:
K=2 # forzar 2 grupos para ver su coincidencia con los grupos según clase de afinidad (fuerte, débil)

kmeans = KMeans(n_clusters=K).fit(X)
centroids = kmeans.cluster_centers_
print(centroids)

[[-1.72149831e+00 -3.05488136e+00 -4.67525424e+00 -6.65310734e+00
  -3.22203390e+01 -1.80336723e+02 -4.02395480e+00 -1.95226893e+02
   1.52254350e+02 -1.52429379e+00  9.51674011e-02 -3.23423729e+00
   1.08384181e+01  2.29378531e+00 -2.56677966e+01  5.42979435e+01
  -3.07344633e+00  6.62146893e+00  8.73107345e+00 -8.91367232e+00
  -4.03028249e+00 -2.82542373e+00 -3.03480226e+02 -6.53143139e-01
  -1.75711864e+01 -1.51777751e+01 -2.44336723e+01 -3.22826260e+01
  -6.43150847e+01 -5.44419209e+01 -5.36500220e+03 -3.93204520e+01
  -2.69294915e+01 -1.19882881e+02 -6.19772316e+01 -1.23695672e+03
  -7.10634407e+02  1.20960452e-01  1.65866667e+01 -7.32598870e+00
  -3.07062147e+00 -1.33050847e+00 -2.24576271e+00 -1.01770795e+02
  -2.63027893e+02 -5.23056045e+01 -5.32605367e+01  2.75627119e-04
   3.97627119e-04 -5.16185706e+01  4.92201582e+01  4.07658983e+01
  -6.36072881e+00  6.50564972e-02 -8.91215819e+00  3.32431638e+00
  -1.89933333e+00 -6.26100000e+00 -1.09884469e+02 -7.47475141e+00
   9.74011

c:\Program Files (x86)\Microsoft Visual Studio\Shared\Python39_64\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [43]:
N = np.shape(X)[0]
for i in range(N):
    X_new =  np.array(X[i]).reshape((1,87)) #complejo 1A2K_C:AB
    #print(X_new)

    new_labels = kmeans.predict(X_new)
    print(new_labels)

[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]


In [45]:
# Predicting the clusters
labels = kmeans.predict(X)

print(labels)
print(y)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 1 1 0 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1 0 0 1 0 1 1 1 1 0 1 1 0 1 0 1 1 1 0
 1 1 0 1 1 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 0 1 1 0 1 0 1 1 0
 1 1 1 0 0 1 1 1 1 1 1 1 0 1 0 0 1 0 1 0 1 1 1 0 0 0 1 1 0 1 1 0 1 1 1 0 1
 1 0 1 0 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 0 0 1 1 1 0 1 1 1 0 1 1 1 0 0 1 1
 1 1 1 0 0 1 0 1 0 0 1 1 0 0 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 0]


In [46]:
K=5 # forzar 2 grupos para ver su coincidencia con los grupos según clase de afinidad (fuerte, débil)

kmeans = KMeans(n_clusters=K).fit(X)
centroids = kmeans.cluster_centers_
print(centroids)

[[-2.71944815e+00 -4.55544444e+00 -6.93185185e+00 -9.10000000e+00
  -4.19629630e+01 -2.27607407e+02 -4.96814815e+00 -2.42854444e+02
   1.87151481e+02 -2.45444444e+00  1.51459741e-01 -5.14629630e+00
   1.45925926e+01  2.45555556e+00 -3.35629630e+01  7.41429259e+01
  -5.00000000e+00  5.73703704e+00  8.68518519e+00 -1.16477778e+01
  -3.95074074e+00 -2.56370370e+00 -3.75381481e+02 -1.06803465e+00
  -2.08000000e+01 -1.81303660e+01 -3.24133333e+01 -4.23870944e+01
  -7.93709259e+01 -6.74477778e+01 -6.99515111e+03 -4.52966667e+01
  -3.06907407e+01 -1.42515741e+02 -7.54988889e+01 -1.55780407e+03
  -9.04090926e+02  1.36703704e+00  1.68809259e+01 -8.68870370e+00
  -3.54629630e+00 -1.24074074e+00 -3.15740741e+00 -1.28199104e+02
  -3.30147814e+02 -5.37528148e+01 -5.39816296e+01  1.84185185e-04
   2.82203704e-04 -6.31203519e+01  6.26627963e+01  5.06834259e+01
  -8.79548148e+00  3.75925926e-02 -1.28231667e+01  7.64933333e+00
  -2.45477778e+00 -7.83350000e+00 -1.28872704e+02 -8.81966667e+00
   1.19074

c:\Program Files (x86)\Microsoft Visual Studio\Shared\Python39_64\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [47]:
# Predicting the clusters
labels = kmeans.predict(X)

print(labels)
print(y)

[3 4 0 4 3 0 2 4 4 0 0 4 4 4 3 4 3 0 4 4 0 3 4 4 0 4 4 4 3 3 4 4 4 4 4 0 4
 4 4 4 0 0 0 0 3 0 4 4 0 3 0 3 3 0 4 4 0 0 0 0 0 0 4 4 4 4 0 3 4 0 4 3 4 4
 4 0 4 0 0 3 4 4 4 4 3 0 4 4 0 4 0 4 4 4 4 0 4 0 0 0 4 4 0 4 0 4 4 0 3 4 0
 4 0 0 3 3 4 2 0 4 4 0 2 4 4 0 4 4 0 0 3 4 4 4 4 4 4 4 1 4 4 0 4 4 4 4 4 4
 4 3 4 0 0 4 0 4 4 3 4 3 0 4 0 4 0 4 0 4 3 4 4 0 4 4 3 4 2 4]
[1 1 1 0 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1 0 0 1 0 1 1 1 1 0 1 1 0 1 0 1 1 1 0
 1 1 0 1 1 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 0 1 1 0 1 0 1 1 0
 1 1 1 0 0 1 1 1 1 1 1 1 0 1 0 0 1 0 1 0 1 1 1 0 0 0 1 1 0 1 1 0 1 1 1 0 1
 1 0 1 0 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 0 0 1 1 1 0 1 1 1 0 1 1 1 0 0 1 1
 1 1 1 0 0 1 0 1 0 0 1 1 0 0 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 0]
